In [1]:
from pyspark.sql import SparkSession

In [14]:
spark.stop()

In [15]:
spark = SparkSession.builder.appName("temp") \
.master("spark://10.208.36.84:7077") \
.config("spark.hadoop.defaultFS","hdfs://10.208.36.84:9000") \
.config("spark.executor.cores",2) \
.config("spark.executor.memory","4g") \
.config("spark.cores.max","10") \
.getOrCreate()


In [9]:
spark

In [16]:
filepath = "/home/tushar/Documents/project/master_file.csv"
df = spark.read.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load(filepath)

In [18]:
df.createOrReplaceTempView('lendingclubdata')

24/03/16 16:34:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [23]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [30]:
from pyspark.sql.functions import sha2,concat_ws

In [31]:
df = df.withColumn('name_sha2',sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state",\
                                                            "grade","sub_grade","verification_status"]),256))

In [32]:
df.select('name_sha2').show()

+--------------------+
|           name_sha2|
+--------------------+
|6d5091b3fcaaeb4ea...|
|b5e7938b0a2da4cea...|
|91060b858433e8a61...|
|cab1fa9f533688b0a...|
|f74e401c1ab0adf78...|
|8aef4bb29d609d8d6...|
|538b4653da3b1e814...|
|b24d55f21390533c5...|
|1035c5401b0ca76d0...|
|cb0f1777593e77909...|
|a962f4d59caec5fa1...|
|e7592ab57b3afd9f1...|
|603afa9d1be879b7b...|
|9fe2d59ddf2a4f37e...|
|23857480ccf555ce4...|
|7c69d5f36fdabf6e1...|
|08bf9e080503b0113...|
|1cafb05aa6c894c30...|
|f2c4010f700d8c9c4...|
|05ad4aed7c393035e...|
+--------------------+
only showing top 20 rows



In [33]:
df.count()

868845

In [34]:
df.select('name_sha2').distinct().count()

868360

In [39]:
df_customer = df.selectExpr("name_sha2 as member_id", "emp_title", "emp_length", "home_ownership", "annual_inc","addr_state", \
"zip_code","'USA' as country", "grade", "sub_grade","verification_status", \
"tot_hi_cred_lim","application_type","annual_inc_joint","verification_status_joint")

In [40]:
df_customer.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/home/tushar/Documents/project/customer_raw") \
.save()

In [43]:
df_loans_data =  df.selectExpr("id as loan_id","name_sha2 as  member_id","loan_amnt","funded_amnt","term","int_rate","installment","issue_d", \
"loan_status","purpose","title")

In [44]:
df_loans_data.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/home/tushar/Documents/project/loans_data_raw") \
.save()

In [46]:
df_lons_repayment = df.selectExpr("id as loan_id","total_rec_prncp","total_rec_int","total_rec_late_fee","total_pymnt",\
                                  "last_pymnt_amnt","last_pymnt_d","next_pymnt_d")

In [47]:
df_lons_repayment.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/home/tushar/Documents/project/loans_repayment_raw") \
.save()

In [48]:
df_loan_deafulter = df.selectExpr("name_sha2 as member_id","delinq_2yrs","delinq_amnt","pub_rec","pub_rec_bankruptcies", \
                                  "inq_last_6mths","total_rec_late_fee","mths_since_last_delinq","mths_since_last_record")

In [49]:
df_loan_deafulter.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/home/tushar/Documents/project/loan_deafulters_raw") \
.save()